In [1]:
!pip install ultralytics


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install Pillow


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
! yolo predict model=yolov8n.pt source='./pra/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg' device=0

Ultralytics YOLOv8.2.29 🚀 Python-3.11.2 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

image 1/1 c:\Users\swara\OneDrive\Desktop\demo\pra\001.Black_footed_Albatross\Black_Footed_Albatross_0001_796111.jpg: 448x640 1 bird, 151.0ms
Speed: 3.2ms preprocess, 151.0ms inference, 79.9ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [4]:
import os
import shutil
import torch
from PIL import Image
from ultralytics import YOLO

In [5]:
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / (1024 ** 2)} MB")
else:
    print("No GPU available.")

GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU
Total Memory: 4095.5 MB


In [6]:
# Path to the main 'images' folder containing all subfolders
images_root_dir = './pra' 
yolo_model = ''
yolo_output_dir = './runs'
result_dir = "./result"
crop_img_path = "./crop_image"

In [7]:
model = YOLO('yolov8n.pt')

In [8]:
# Function to run YOLO command on each image
def run_yolo(image_path):
    result = model(image_path, save = True)
    x1, y1, x2, y2 = result[0].boxes.xyxy.int().tolist()[0]
    bounding_box = (x1, y1, x2, y2)
    return bounding_box

In [9]:
#crop image function
def crop_and_save_img(image_path, bounding_box, file):
    with Image.open(image_path) as img:
        cropped_img = img.crop(bounding_box)
        cropped_img
    cropped_image = Image.fromarray(cropped_img)
    output_path = crop_img_path + "/"+ +"/"+file  # Specify your desired output path
    cropped_image.save(output_path)

In [10]:
# Function to move result image back to the original folder
def move_result_image(original_image_path, result_image_folder, file):
    # Get the original folder path and file name
    original_folder = os.path.dirname(original_image_path)
    result_folder = result_dir + original_folder[5:]
    result_image_path = os.path.join(result_image_folder, os.listdir(result_image_folder)[0])
    # Move the result image to the original folder
    shutil.move(result_image_path, result_folder)

# Function to delete the 'runs/detect/predict' folder
def clean_up_yolo_output():
    if os.path.exists(yolo_output_dir):
        shutil.rmtree(yolo_output_dir)

In [11]:
# Main function to process all images
def process_images():
    for subdir, _, files in os.walk(images_root_dir):
        for file in files:
            # Only process image files (assuming they have jpg or png extensions)
            if file.endswith(('.jpg', '.png', '.jpeg')):
                image_path = os.path.join(subdir, file)
                # print(f"{subdir} {file} {image_path}")
                # Run YOLO on the image
                bounding_box = run_yolo(image_path)

                #crop the image
                crop_and_save_img(image_path, bounding_box)

                # Move the result image to the original folder
                move_result_image(image_path, "./runs/detect/predict", file)    

                # Clean up the YOLO output folder
                clean_up_yolo_output()
                break

In [12]:
process_images()